In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import pickle 

with open("train_test_split.pkl", 'rb') as f:
     X_train_tfidf, X_test_tfidf, y_train_enc, y_test_enc = pickle.load(f)

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential

In [5]:
X_test_tfidf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 722 stored elements and shape (147, 600)>

Here the X_train, X_test are in sparse so we need to conver to dense by toarray

In [65]:
X_train_dense = X_train_tfidf.toarray()
X_test_dense = X_test_tfidf.toarray()

model = Sequential([
    Dense(128, activation='relu', input_dim=X_train_tfidf.shape[1]),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(5, activation='softmax')
])

c:\Users\Bijay\Downloads\DJANGO_PROJECTS\ML\codveda\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


If we had binary target then sigmoid so for now softmax works for 5 classes

In [66]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [67]:
model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_27 (Dense)                │ (None, 128)            │        76,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 85,509 (334.02 KB)

 Trainable params: 85,509 (334.02 KB)

 Non-trainable params: 0 (0.00 B)

In [68]:
history = model.fit(X_train_dense, y_train_enc,
                    epochs=50,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5064 - loss: 1.5306 - val_accuracy: 0.4359 - val_loss: 1.4951
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5406 - loss: 1.3709 - val_accuracy: 0.4359 - val_loss: 1.4046
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5406 - loss: 1.2114 - val_accuracy: 0.4359 - val_loss: 1.3959
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5406 - loss: 1.1318 - val_accuracy: 0.4359 - val_loss: 1.3704
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5406 - loss: 1.0442 - val_accuracy: 0.4359 - val_loss: 1.3215
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5406 - loss: 0.9650 - val_accuracy: 0.4359 - val_loss: 1.2852
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5876 - loss: 0.8663 - val_accuracy: 0.4786 - val_loss: 1.2432
Epoch 8/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6795 - loss: 0.7651 - val_accuracy: 0.5470 - val_loss

In [69]:
y_pred_nn = model.predict(X_test_dense)
y_pred_classes = np.argmax(y_pred_nn, axis=1)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

accuracy = accuracy_score(y_test_enc, y_pred_classes)

print(f"Accuracy: {accuracy:.4f}", )
print("\nConfusion Matrix:\n", confusion_matrix(y_test_enc, y_pred_classes))
print("\nClassification Report:\n", classification_report(y_test_enc, y_pred_classes))
 

Accuracy: 0.7687

Confusion Matrix:
 [[20  1  4  0  0]
 [ 1 12 10  0  1]
 [ 3  4 68  0  1]
 [ 1  0  2 10  0]
 [ 0  1  5  0  3]]

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.80      0.80        25
           1       0.67      0.50      0.57        24
           2       0.76      0.89      0.82        76
           3       1.00      0.77      0.87        13
           4       0.60      0.33      0.43         9

    accuracy                           0.77       147
   macro avg       0.77      0.66      0.70       147
weighted avg       0.77      0.77      0.76       147

